<a href="https://colab.research.google.com/github/LeeSeungwon89/Deep-learning_Theory/blob/main/9-2%20%EC%88%9C%ED%99%98%20%EC%8B%A0%EA%B2%BD%EB%A7%9D%EC%9C%BC%EB%A1%9C%20IMDB%20%EB%A6%AC%EB%B7%B0%20%EB%B6%84%EB%A5%98%ED%95%98%EA%B8%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **9-2 순환 신경망으로 IMDB 리뷰 분류하기**

IMDB 리뷰 데이터 세트로 간단한 순환 신경망 모델을 훈련해 보겠습니다. 이 데이터 세트를 두 가지 방법(원-핫 인코딩, 단어 임베딩)으로 변형하여 순환 신경망에 주입해 보겠습니다.

## **IMDB 리뷰 데이터셋**

IMDB 리뷰 데이터 세트는 인터넷 영화 데이터베이스인 imdb.com 사이트에서 수집한 리뷰를 긍정과 부정으로 분류한 데이터 세트입니다. 샘플 50,000개로 구성되며 훈련 데이터와 테스트 데이터는 25,000개씩 구성됩니다.

**자연어 처리(natural language processing, NLP)**는 컴퓨터를 사용하여 인간의 언어(자연어)를 처리하는 분야입니다. 대표되는 세부 분야는 음성 인식, 기계 번역, 감성 분석 등입니다. 이 챕터에서 다룰 데이터 세트로는 감성 분석을 수행합니다. 자연어 처리 분야에서는 훈련 데이터를 **말뭉치(corpus)**라고 부르기도 합니다.

기실 텍스트 자체를 신경망에 전달하는 것이 아닙니다. 컴퓨터는 숫자 데이터만 처리할 수 있습니다. 합성곱 신경망에서 이미지를 다룰 때 이미지는 이미 픽셀값(정수)으로 구성되므로 숫자로 변환하는 작업이 필요하지 않습니다. 그러나 텍스트 데이터는 단어를 정수 데이터로 변환하는 작업이 필요합니다. 아래와 같은 문장이 있다고 가정하겠습니다.

He follows the cat. He loves the cat.

이 문장의 단어인 He, follows, the, cat. 등에 고유한 정수를 부여(매핑)합니다. 같은 단어에는 같은 정수를 부여합니다. 정수 사이에는 어떠한 관계도 없이 고유할 뿐입니다. 일반적으로 영어 문장은 모두 소문자로 바꾸고 구둣점을 삭제하여 공백 기준으로 분리합니다. 이렇게 분리된 단어를 **토큰(token)**이라고 부릅니다. 샘플 하나는 여러 토큰으로 구성되며, 토큰 1개는 타임스템프 하나입니다. 간단한 문제라면 영어 말뭉치에서 토큰을 단어와 같다고 볼 수 있습니다. 토큰에 할당하는 정수 중에 몇 개는 특정 용도로 예약되어 있습니다. 예컨대 0은 패딩, 1은 문장 시작, 2는 어휘 사전(훈련 데이터 세트에서 고유한 단어를 뽑아 생성한 목록)에 존재하지 않는 토큰을 의미합니다. 테스트 데이터 세트에 어휘 사전에 존재하지 않는 단어가 있다면 2로 변환하여 신경망 모델에 주입합니다. 

참고로 한국어는 영어와 전혀 다릅니다. 한국어는 조사가 발달된 언어이므로 공백으로만 나눠서는 문장 의미를 제대로 분석하기 어렵습니다. 한국어에 대한 자연어 처리는 `KoNLPy`를 사용합니다.

IMDB 리뷰 데이터 세트는 영어로 된 문장입니다. 텐서플로에 이 데이터 세트를 정수로 바꾼 데이터가 포함되어 있으므로 굳이 정수로 바꾸는 작업은 필요하지 않습니다. 데이터를 준비하겠습니다.

In [2]:
from tensorflow.keras.datasets import imdb

# 가장 자주 등장하는 단어 500개만 사용하기 위해 `num_words=500`으로 지정합니다.
(train_input, train_target), (test_input, test_target) = imdb.load_data(num_words=500)
print(train_input.shape, test_input.shape)

17473536/17464789 [==============================] - 0s 0us/step
(25000,) (25000,)


처음 데이터만 확인해 보겠습니다.

In [7]:
print(train_input)

[list([1, 14, 22, 16, 43, 2, 2, 2, 2, 65, 458, 2, 66, 2, 4, 173, 36, 256, 5, 25, 100, 43, 2, 112, 50, 2, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 2, 2, 17, 2, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2, 19, 14, 22, 4, 2, 2, 469, 4, 22, 71, 87, 12, 16, 43, 2, 38, 76, 15, 13, 2, 4, 22, 17, 2, 17, 12, 16, 2, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2, 2, 16, 480, 66, 2, 33, 4, 130, 12, 16, 38, 2, 5, 25, 124, 51, 36, 135, 48, 25, 2, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 2, 15, 256, 4, 2, 7, 2, 5, 2, 36, 71, 43, 2, 476, 26, 400, 317, 46, 7, 4, 2, 2, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2, 56, 26, 141, 6, 194, 2, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 2, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 2, 88, 12, 16, 283, 5, 16, 2, 113, 103, 32, 15, 16, 2, 19, 178, 32])
 list([1, 194, 2, 194, 2, 78, 228, 5, 6, 2, 2, 2, 134, 26, 4, 2, 8, 118, 2, 14, 394, 20, 13, 119, 2, 189, 102, 5, 207, 110, 2, 21, 14, 69, 188, 

리뷰 텍스트는 길이가 모두 다르므로 2차원 배열에 담지 않고 리뷰마다 별도로 파이썬 리스트로 담아야 메모리를 효율적으로 사용할 수 있습니다. 따라서 리스트 안에 리스트(리뷰 하나) 객체가 여러 개 나열된 형태이며 1차원 넘파이 배열입니다.

리뷰 몇 개만 길이를 확인해 보겠습니다.

In [10]:
print(len(train_input[0]))
print(len(train_input[1]))
print(len(train_input[2]))

218
189
141


각각 토큰 218개, 189개, 141개로 구성된 리뷰입니다. 리뷰 하나는 샘플 하나입니다.

첫 번째 리뷰 내용만 확인해 보겠습니다.

In [12]:
print(train_input[0])

[1, 14, 22, 16, 43, 2, 2, 2, 2, 65, 458, 2, 66, 2, 4, 173, 36, 256, 5, 25, 100, 43, 2, 112, 50, 2, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 2, 2, 17, 2, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2, 19, 14, 22, 4, 2, 2, 469, 4, 22, 71, 87, 12, 16, 43, 2, 38, 76, 15, 13, 2, 4, 22, 17, 2, 17, 12, 16, 2, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2, 2, 16, 480, 66, 2, 33, 4, 130, 12, 16, 38, 2, 5, 25, 124, 51, 36, 135, 48, 25, 2, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 2, 15, 256, 4, 2, 7, 2, 5, 2, 36, 71, 43, 2, 476, 26, 400, 317, 46, 7, 4, 2, 2, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2, 56, 26, 141, 6, 194, 2, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 2, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 2, 88, 12, 16, 283, 5, 16, 2, 113, 103, 32, 15, 16, 2, 19, 178, 32]


정수로 변환된 리뷰입니다. 위에서 `num_words=500`으로 지정했으므로 어휘 사전에는 단어가 500개만 구성된 상태이며, 어휘 사전에 없는 단어는 2로 대체됐습니다.

타깃 데이터를 확인해 보겠습니다.

In [13]:
print(train_target[:20])

[1 0 0 1 0 0 1 0 1 0 1 0 0 0 0 0 1 1 0 1]


부정은 0, 긍정은 1이며 이진 분류 문제입니다.

훈련 데이터 세트에서 검증 데이터 세트를 만들겠습니다.

In [14]:
from sklearn.model_selection import train_test_split

train_input, val_input, train_target, val_target = train_test_split(
    train_input, train_target, test_size=0.2, random_state=42)



---

작업 중

---



## **순환 신경망 만들기**

## **순환 신경망 훈련하기**

## **단어 임베딩을 사용하기**